# 34. notMNIST CNN

<p style="text-align: right;">
blackdew7@gmail.com<br>
Your name :
</p>


#### 선행지식
1. TensorFlow 다루기 기초
2. 모델링을 한다는 것에 대한 이해.
3. Supervised Learning 중 Classification에 대한 기본 지식.
4. ANN에 대한 기본 구조와 개념

#### 실습목표
1. ANN의 구조를 Graph로 그려낼 수 있다.
2. 그려낸 Graph를 텐서플로우를 이용해 코딩할 수 있다.

#### 사용데이터.

01. Multinomial Classification : http://yann.lecun.com/exdb/mnist/

## 00. 이미지 로더

In [0]:
from scipy.misc import imread, imresize
from sklearn.model_selection import train_test_split

import os
import numpy as np
import matplotlib.pyplot as plt

class ImageLoader:
    def __init__(self, path, folders):
        images, labels = [], []

        for i, folder_name in enumerate(folders):
            directory = path + folder_name + '/'
            files = os.listdir(directory)
            label = np.array([0] * 10)
            label[i] = 1
            for file in files:
                try:
                    im = imread(directory + file)
                except:
                    print("Skip a corrupted file: " + file)
                    continue
                pixels = np.array(im.astype(float))
                images.append(pixels / 255.0)
                labels.append(label)

        train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=0)
        
        class train:
            def __init__(self):
                self.images = []
                self.labels = []
                self.batch_counter = 0
                self.num_examples = 0

            def get_num_examples(self):
                    return len(self.images)

            def next_batch(self, num):
                if self.batch_counter + num >= len(self.labels):
                    batch_images = self.images[self.batch_counter:]
                    batch_labels = self.labels[self.batch_counter:]
                    left = num - len(batch_labels)
                    batch_images.extend(self.images[:left])
                    batch_labels.extend(self.labels[:left])
                    self.batch_counter = left
                else:
                    batch_images = self.images[self.batch_counter:self.batch_counter+num]
                    batch_labels = self.labels[self.batch_counter:self.batch_counter+num]                  
                    self.batch_counter += num
                    
                return (batch_images, batch_labels)
                    
        class test:
            def __init__(self):
                self.images = []
                self.labels = []
                
        self.train = train()
        self.test = test()

        self.train.images = train_images
        self.train.labels = train_labels
        self.test.images = test_images
        self.test.labels = test_labels

#### 파일 업로드 
notMNIST_small.tar.gz

In [0]:
# file update
from google.colab import files
uploaded = files.upload() # then browse, select the files. It's then uploaded

In [0]:
# notMNIST_small.tar.gz 압축해제
import sys
import tarfile

tar = tarfile.open('./notMNIST_small.tar.gz')
sys.stdout.flush()
tar.extractall('./')
tar.close()

In [0]:
!ls -al

#### 이미지 로딩

In [0]:
mnist = ImageLoader('./notMNIST_small/', ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])

#### 이미지 로딩 테스트

In [0]:
print(len(mnist.train.images))   # Images in the training set.
print(len(mnist.train.labels))   # Labels in the training set.
print(len(mnist.test.images))    # Images in the test set.
print(len(mnist.test.labels))    # Labels in the test set.

train, label = mnist.train.next_batch(100) # To retrieve specified number of data in a batch

print(np.array(train).shape)
print(np.array(label).shape)

In [0]:
'''
mnist 시각화 함수
'''
def mnist_plot(i):
    pixels = mnist.test.images[i].reshape((28, 28))
    a = mnist.test.labels
    plt.title('Label : {}'.format(a[i]))
    plt.imshow(pixels, cmap='gray')
    return plt.show()

In [0]:
'''
반복실행해보자
'''
mnist_plot(np.random.randint(0, high=3745))

# ANN 실습

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

# 데이터 셋을 로딩하여 train, test 데이터를 만들자 
mnist = ImageLoader('./notMNIST_small/', ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])

In [0]:
# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()

# Hyper-Parameters & Option
learning_rate = 0.01

# Input Layer, Real Y
X = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28])
X_img = tf.reshape(X, [-1, 28 * 28])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

# 1st Hidden Layer
W1 = tf.Variable(tf.random_normal([28 * 28, 512]))
b1 = tf.Variable(tf.random_normal([512]))
sum1 = tf.add(tf.matmul(X_img, W1), b1)
hidden_output1 = tf.nn.relu(sum1)

# Output Layer
Wo = tf.Variable(tf.random_normal([512, 10]))
bo = tf.Variable(tf.random_normal([10]))
logits = tf.add(tf.matmul(hidden_output1, Wo), bo)
Y_prob = tf.nn.softmax(logits)

# Classification & Accuracy
n_of_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(n_of_correct, tf.float32))

# Cost(loss) function & Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



In [0]:
sess = tf.Session()
tf.set_random_seed(2017)
sess.run(tf.global_variables_initializer())

training_epochs = 5
batch_size = 128
n_of_batches = int(len(mnist.train.images) / batch_size)

for i in range(training_epochs + 1):
    print("{}th epoch".format(i))

    for batch in range(n_of_batches):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})
        
        if (batch % 10 == 0):
            tr_a, loss = sess.run([accuracy, cost], feed_dict={X: X_batch, Y: Y_batch})
            print("training loss: {:.4f} accuracy: {:.2f}%".format(loss, tr_a * 100))

print("Training Complete")

In [0]:
# 완성된 모델을 검증하는 코드를 작성하자
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))
# print(sess.run([W[:, 0], b]))
# sess.close()


In [0]:
n = int(np.random.uniform(0, len(mnist.test.images), 1))
Predicted_distribution = sess.run(Y_prob, feed_dict={X: [mnist.test.images[n]]})
Predicted_distribution = Predicted_distribution[0]
pd_dict = {i: '%.2f' % prob for i, prob in enumerate(Predicted_distribution)}

print(n)
print(pd_dict)
mnist_plot(n)


In [0]:
# 세션을 종료하자
sess.close()